In [8]:
# Cell 0: Find Available Models (FIXED)
import requests
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('GEMINI_API_KEY')

url = "https://generativelanguage.googleapis.com/v1beta/models"

response = requests.get(f"{url}?key={api_key}")

if response.status_code == 200:
    models = response.json()
    
    print("✅ Available Models:\n")
    for model in models.get('models', []):
        model_name = model.get('name', '')
        methods = model.get('supportedGenerationMethods', [])
        
        if 'generateContent' in methods:
            print(f"✅ {model_name}")
    
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

✅ Available Models:

✅ models/gemini-2.5-pro-preview-03-25
✅ models/gemini-2.5-flash
✅ models/gemini-2.5-pro-preview-05-06
✅ models/gemini-2.5-pro-preview-06-05
✅ models/gemini-2.5-pro
✅ models/gemini-2.0-flash-exp
✅ models/gemini-2.0-flash
✅ models/gemini-2.0-flash-001
✅ models/gemini-2.0-flash-exp-image-generation
✅ models/gemini-2.0-flash-lite-001
✅ models/gemini-2.0-flash-lite
✅ models/gemini-2.0-flash-lite-preview-02-05
✅ models/gemini-2.0-flash-lite-preview
✅ models/gemini-2.0-pro-exp
✅ models/gemini-2.0-pro-exp-02-05
✅ models/gemini-exp-1206
✅ models/gemini-2.0-flash-thinking-exp-01-21
✅ models/gemini-2.0-flash-thinking-exp
✅ models/gemini-2.0-flash-thinking-exp-1219
✅ models/gemini-2.5-flash-preview-tts
✅ models/gemini-2.5-pro-preview-tts
✅ models/learnlm-2.0-flash-experimental
✅ models/gemma-3-1b-it
✅ models/gemma-3-4b-it
✅ models/gemma-3-12b-it
✅ models/gemma-3-27b-it
✅ models/gemma-3n-e4b-it
✅ models/gemma-3n-e2b-it
✅ models/gemini-flash-latest
✅ models/gemini-flash-lite-lat

In [9]:
# Cell 1: Setup
print("💬 INGRES Chatbot Test")
print("="*60)

import sys
sys.path.append('..')

import google.generativeai as genai
from chromadb import PersistentClient
import os
from dotenv import load_dotenv

# Load API key
load_dotenv()

print("✅ Imports successful!")

💬 INGRES Chatbot Test
✅ Imports successful!


In [10]:
# Cell 2: Initialize (Simplified)
print("🤖 Initializing...\n")

# Get API key
api_key = os.getenv('GEMINI_API_KEY')

if not api_key or api_key == 'your_key_will_go_here':
    print("❌ ERROR: No API key!")
else:
    print(f"✅ API key loaded: {api_key[:20]}...")
    
    # Load vector store
    client = PersistentClient(path="../data/embeddings")
    collection = client.get_collection("groundwater_data")
    print(f"✅ Vector store loaded ({collection.count()} documents)")

🤖 Initializing...

✅ API key loaded: AIzaSyAl3xiwoF2rdyuw...
✅ Vector store loaded (27 documents)


In [11]:
# Cell 3: Chat Function (With Rate Limit Protection)
import requests
import time

def ask_chatbot(question):
    """Ask the chatbot using direct API call"""
    
    print(f"\n💭 YOU: {question}")
    print("="*60)
    
    # Search for relevant data
    print("🔍 Searching database...")
    results = collection.query(
        query_texts=[question],
        n_results=5
    )
    
    # Get context
    context = "\n\n".join(results['documents'][0])
    
    # Create prompt
    prompt = f"""You are an expert assistant for INGRES (India Groundwater Resource Estimation System).

Context from database:
{context}

User Question: {question}

Provide a clear, accurate answer based on the context. Include specific numbers and district names.

Categories:
- Safe: Stage of Extraction < 70%
- Semi-Critical: 70-90%
- Critical: 90-100%
- Over-Exploited: > 100%

Answer:"""
    
    # Get AI response
    print("🤔 Generating answer...")
    
    url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"
    
    headers = {
        'Content-Type': 'application/json',
    }
    
    data = {
        "contents": [{
            "parts": [{
                "text": prompt
            }]
        }]
    }
    
    # Try up to 3 times if rate limited
    max_retries = 3
    for attempt in range(max_retries):
        response = requests.post(
            f"{url}?key={api_key}",
            headers=headers,
            json=data
        )
        
        if response.status_code == 200:
            result = response.json()
            answer = result['candidates'][0]['content']['parts'][0]['text']
            
            print("\n🤖 CHATBOT:")
            print("="*60)
            print(answer)
            print("="*60)
            
            return answer
            
        elif response.status_code == 503:
            if attempt < max_retries - 1:
                wait_time = 5 * (attempt + 1)
                print(f"⏳ Rate limited. Waiting {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"❌ Error 503: Service temporarily unavailable")
                print("Try again in a minute!")
                return None
        else:
            print(f"❌ Error: {response.status_code}")
            print(response.text)
            return None

print("✅ Chat function ready!")

✅ Chat function ready!


In [12]:
# Cell 4: Ask about Guntur
ask_chatbot("What is the stage of groundwater development in Guntur district?")


💭 YOU: What is the stage of groundwater development in Guntur district?
🔍 Searching database...
🤔 Generating answer...

🤖 CHATBOT:
In Guntur district, the stage of groundwater development is **36.0%**.

Based on the provided categories, Guntur district is classified as **Safe** (Stage of Extraction < 70%).


'In Guntur district, the stage of groundwater development is **36.0%**.\n\nBased on the provided categories, Guntur district is classified as **Safe** (Stage of Extraction < 70%).'

In [13]:
# Cell 5: Ask about over-exploited districts
ask_chatbot("Which districts are over-exploited?")


💭 YOU: Which districts are over-exploited?
🔍 Searching database...
🤔 Generating answer...
⏳ Rate limited. Waiting 5 seconds...
⏳ Rate limited. Waiting 10 seconds...

🤖 CHATBOT:
Based on the provided context and categories, none of the districts listed are classified as over-exploited (Stage of Ground Water Development > 100%).

Here's the breakdown for each district:
*   **Karimnagar**: Stage of Ground Water Development (%) is 70.0%, which falls into the **Semi-Critical** category (70-90%).
*   **Vizianagaram**: Stage of Ground Water Development (%) is 79.0%, which falls into the **Semi-Critical** category (70-90%).
*   **District: 2**: Stage of Ground Water Development (%) is 15.0%, which falls into the **Safe** category (< 70%).
*   **Nalgonda**: Stage of Ground Water Development (%) is 69.0%, which falls into the **Safe** category (< 70%).
*   **Nellore**: Stage of Ground Water Development (%) is 59.0%, which falls into the **Safe** category (< 70%).


"Based on the provided context and categories, none of the districts listed are classified as over-exploited (Stage of Ground Water Development > 100%).\n\nHere's the breakdown for each district:\n*   **Karimnagar**: Stage of Ground Water Development (%) is 70.0%, which falls into the **Semi-Critical** category (70-90%).\n*   **Vizianagaram**: Stage of Ground Water Development (%) is 79.0%, which falls into the **Semi-Critical** category (70-90%).\n*   **District: 2**: Stage of Ground Water Development (%) is 15.0%, which falls into the **Safe** category (< 70%).\n*   **Nalgonda**: Stage of Ground Water Development (%) is 69.0%, which falls into the **Safe** category (< 70%).\n*   **Nellore**: Stage of Ground Water Development (%) is 59.0%, which falls into the **Safe** category (< 70%)."

In [14]:
# Cell 6: Compare districts
ask_chatbot("Compare groundwater availability between Krishna and Vizianagaram districts")


💭 YOU: Compare groundwater availability between Krishna and Vizianagaram districts
🔍 Searching database...
🤔 Generating answer...
⏳ Rate limited. Waiting 5 seconds...
⏳ Rate limited. Waiting 10 seconds...

🤖 CHATBOT:
Comparing groundwater availability between Krishna and Vizianagaram districts:

*   **Vizianagaram district** has a Net Ground Water Availability of **144156.0** units, with a Stage of Ground Water Development of **79.0%**. This places Vizianagaram in the **Semi-Critical** category.
*   **Krishna district** has a Net Ground Water Availability of **118668.0** units, with a Stage of Ground Water Development of **83.0%**. This also places Krishna in the **Semi-Critical** category.

Therefore, Vizianagaram district has a higher Net Ground Water Availability (144156.0 units) compared to Krishna district (118668.0 units), although both districts are classified as Semi-Critical in terms of groundwater development.


'Comparing groundwater availability between Krishna and Vizianagaram districts:\n\n*   **Vizianagaram district** has a Net Ground Water Availability of **144156.0** units, with a Stage of Ground Water Development of **79.0%**. This places Vizianagaram in the **Semi-Critical** category.\n*   **Krishna district** has a Net Ground Water Availability of **118668.0** units, with a Stage of Ground Water Development of **83.0%**. This also places Krishna in the **Semi-Critical** category.\n\nTherefore, Vizianagaram district has a higher Net Ground Water Availability (144156.0 units) compared to Krishna district (118668.0 units), although both districts are classified as Semi-Critical in terms of groundwater development.'

In [15]:
# Cell 7: Your custom question
your_question = "What is the total annual recharge in Andhra Pradesh?"

ask_chatbot(your_question)


💭 YOU: What is the total annual recharge in Andhra Pradesh?
🔍 Searching database...
🤔 Generating answer...

🤖 CHATBOT:
Based on the provided context for the districts currently within Andhra Pradesh, the total annual replenishable groundwater resource is **306,758**.

Here's the breakdown by district:
*   **Anantapur**: 116,570
*   **Srikakulam**: 48,619
*   **Krishna**: 72,443
*   **Vizianagaram**: 69,126

(Please note: The district of Karimnagar, with an Annual Replenishable Ground Water Resource of 43,753, is located in Telangana and has therefore been excluded from the total for Andhra Pradesh.)


"Based on the provided context for the districts currently within Andhra Pradesh, the total annual replenishable groundwater resource is **306,758**.\n\nHere's the breakdown by district:\n*   **Anantapur**: 116,570\n*   **Srikakulam**: 48,619\n*   **Krishna**: 72,443\n*   **Vizianagaram**: 69,126\n\n(Please note: The district of Karimnagar, with an Annual Replenishable Ground Water Resource of 43,753, is located in Telangana and has therefore been excluded from the total for Andhra Pradesh.)"

In [ ]:
# Cell 8: Interactive Mode
print("💬 Interactive Chat Mode")
print("="*60)
print("Ask questions! Type 'quit' to stop.\n")

while True:
    question = input("YOU: ").strip()
    
    if question.lower() in ['quit', 'exit', 'bye', 'stop']:
        print("\n👋 Goodbye!")
        break
    
    if not question:
        continue
    
    ask_chatbot(question)

💬 Interactive Chat Mode
Ask questions! Type 'quit' to stop.

